In [ ]:
from pathlib import Path
from functools import partial

import numpy as np
from scipy import constants as C
import ipywidgets as widgets

from mmwave.rma import rma, echo_plot
from mmwave.util import load_frame

inslider_style = {"description_width": "initial", "width": "50%"}
inslider_layout = widgets.Layout(width="50%")

IntSlider = partial(widgets.IntSlider, style=inslider_style, layout=inslider_layout)

In [ ]:
output_dir = Path("/home/tomin/python/mmwave_postproc/cast_data/small_gun_241121")

frame_file, cfg = load_frame(output_dir)

num_sample = cfg.mimo.profile.numAdcSamples
adcStartTime = cfg.mimo.profile.adcStartTime  # us
startFrequency = cfg.mimo.profile.startFrequency * 1e9
K = cfg.mimo.profile.frequencySlope * 1e12  # Slope const (hz/s)
Fs = cfg.mimo.profile.adcSamplingFrequency * 1e3  # Sampling rate (sps)

F0 = startFrequency + adcStartTime * K * 1e-6 + num_sample // 2 / Fs * K  # Center frequency

dx = cfg.bracket.profile.dx
dy = cfg.bracket.profile.dy  # Sampling distance at x (horizontal) y (vertical) axis in mm
row = cfg.bracket.profile.row
col = cfg.bracket.profile.col

c = C.c
Ts = 1 / Fs  # Sampling period
k = 2 * np.pi * F0 / c  # Wave number
print(f"k:{k}")

In [ ]:
tx_idx = IntSlider(value=0, min=0, max=11, step=1, description="发天线")
rx_idx = IntSlider(value=0, min=0, max=15, step=1, description="收天线")
ID_select = IntSlider(value=21, min=0, max=frame_file.shape[4] - 1, step=1, description="成像距离")
nFFTtime = num_sample  # Number of FFT points for Spatial-FFT
tI = 183  # mm


@widgets.interact(tx_idx=tx_idx, rx_idx=rx_idx, ID_select=ID_select)
def rma_interact(tx_idx, rx_idx, ID_select):
    R = c / 2 * (ID_select / (K * Ts * nFFTtime)) - tI / 1000
    Echo = frame_file[rx_idx, tx_idx, :, :, :, 0] + 1j * frame_file[rx_idx, tx_idx, :, :, :, 1]
    Sr: np.ndarray = np.fft.fft(Echo)
    Sr = Sr[:, :, ID_select - 1]
    Sr[Sr == 0] = 1e-10
    # echo_plot(Sr, "source", dx, dy)

    reconstructed_image = rma(Sr, dx, dy, R, k)
    echo_plot(reconstructed_image, "reconstructed_image", dx, dy, rma=True)